In [14]:
import numpy as np
import pandas as pd
hz500_rawecg = np.load('ecg_np_to_extract_500hz.npy')
log_500hz = pd.read_excel('500hz_set_ecg_selected.xlsx')
hz250_rawecg = np.load('ecg_np_to_extract_250hz.npy')
log_250hz = pd.read_excel('250hz_set_ecg_selected.xlsx')

print(hz500_rawecg.shape)
print(hz250_rawecg.shape)
print(log_500hz.shape)
print(log_250hz.shape)

a = log_500hz['MDN']
b = log_250hz['MDN']
print("The number of duplicate ECG's", len(set(a) & set(b)))
print(len(log_250hz.StudyID_x.unique()))
print(len(log_500hz.StudyID_x.unique()))

(1153, 5000, 12)
(941, 2500, 12)
(1153, 291)
(941, 291)
The number of duplicate ECG's 295
941
1153


In [18]:
hz500_rawecg = np.load('ecg_np_to_extract_500hz.npy')
hz500_rawecg_1 = np.load('ecg_np_to_extract_500hz_1.npy')
hz250_rawecg = np.load('ecg_np_to_extract_250hz.npy')
hz250_rawecg_1 = np.load('ecg_np_to_extract_250hz_1.npy')


In [21]:
print(hz500_rawecg[5])
print(hz500_rawecg_1[5])

[[-20.877842  -10.992804    8.061044  ... -23.280523  -20.58204
   -5.7699666]
 [-21.647009  -10.757448    8.533437  ... -22.010015  -18.962978
   -5.0327215]
 [-22.00366   -11.1311      8.108554  ... -21.190834  -17.876516
   -4.821511 ]
 ...
 [-45.11135   -54.75475    63.59275   ... -29.794239  -38.379787
  -85.153145 ]
 [-44.766266  -54.219208   79.95553   ... -28.274712  -34.79672
  -82.23408  ]
 [-44.453564  -53.604572   96.41739   ... -27.088444  -31.506676
  -79.17096  ]]
[[-20.877842  -10.992804    8.061044  ... -23.280523  -20.58204
   -5.7699666]
 [-21.647009  -10.757448    8.533437  ... -22.010015  -18.962978
   -5.0327215]
 [-22.00366   -11.1311      8.108554  ... -21.190834  -17.876516
   -4.821511 ]
 ...
 [-45.11135   -54.75475    63.59275   ... -29.794239  -38.379787
  -85.153145 ]
 [-44.766266  -54.219208   79.95553   ... -28.274712  -34.79672
  -82.23408  ]
 [-44.453564  -53.604572   96.41739   ... -27.088444  -31.506676
  -79.17096  ]]


In [16]:
#Down sample the 500hz to 250hz
hz500_rawecg_convert = []

from scipy import signal
for a in hz500_rawecg:
    f = signal.resample(a, 2500)
    hz500_rawecg_convert.append(f)

hz500_rawecg_convert = np.array(hz500_rawecg_convert)

#Stack both ECG datasets
test = np.concatenate((hz250_rawecg, hz500_rawecg_convert), axis=0)
np.save('combined_rawecg_downsample.npy', test) # save

#Stack the datasets
test_log = pd.concat([log_250hz, log_500hz], axis=0)
test_log.to_excel('combined_log_downsampled.xlsx') 

#Check that the numbers are equal
print(test.shape)
print(test_log.shape)

#Get the rownumbers of duplicate cases
test_log.reset_index(drop=True, inplace=True)
pd.set_option('display.max_rows', 300)
test_log.to_excel('combined_log_downsampled.xlsx')  
test_log.index.groupby(test_log['StudyID_x'])
idx = test_log
flas = idx['StudyID_x'].drop_duplicates(keep='first')

#Make list of duplicates
keep_ecg = flas.index.values.tolist()

#ECGs to keep
idx = idx.iloc[keep_ecg]
idx.to_excel('selected_dataset_downsampled_final.xlsx')  

# #Extract ECGs from HDF5 corresponding to patients (around implantation)
axis = 0
ecg_np_to_extract = np.take(test, keep_ecg, axis)
print(ecg_np_to_extract.shape)
np.save('selected_ecg_downsampled_final.npy', ecg_np_to_extract) # save

(2094, 2500, 12)
(2094, 291)
(1799, 2500, 12)
